# Cargar librerias

In [1]:
import pandas as pd
import numpy as np
import openpyxl
import os
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime, timedelta

# Cargar bases de datos 

In [2]:
# Configuración de pandas para mostrar más dígitos en la salida
pd.set_option('display.float_format', '{:.22f}'.format)

In [3]:
# Imprime el directorio actual
print("Directorio inicial:", os.getcwd())

# Cambiar al directorio 
os.chdir('..')

# Confirma el cambio de directorio
print("Directorio después de moverse hacia arriba:", os.getcwd())

Directorio inicial: D:\OneDrive - Allianz\6_Procesos TRANSMISIONES\FORMATOS 253-251\Codigo
Directorio después de moverse hacia arriba: D:\OneDrive - Allianz\6_Procesos TRANSMISIONES\FORMATOS 253-251


# TXT

In [6]:
# Ruta a los archivos Excel
archivo_251_path = r"D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATOS 253-251/Bases/Plantilla_251.xlsx"
archivo_253_path = r"D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATOS 253-251/Bases/Plantilla_253.xlsx"

# Definir los nombres de las columnas manualmente para el formato 251
columnas_251 = ['SUBCUENTA', 'CONCEPTO', 'HOMBRES', 'MUJERES', 'HOMBRES_2', 'MUJERES_2', 'TOTAL', 'UNIDAD DE CAPTURA']

# Cargar los datos de la plantilla 251
Plantilla_251 = pd.read_excel(archivo_251_path, header=None, usecols="A:H", nrows=15, names=columnas_251)

# Definir las columnas clave para el formato 253
columnas_253 = ['SUBCUENTA', 'CONCEPTO', 'VALOR', 'UNIDAD DE CAPTURA']

# Cargar los datos de la plantilla 253
Plantilla_253 = pd.read_excel(archivo_253_path, header=3, usecols="A:D", names=columnas_253)

# Definir las columnas clave
columna_subcuenta = 'SUBCUENTA'
columna_unidad_captura = 'UNIDAD DE CAPTURA'

# Definimos la función para formatear las líneas
def formatear_linea(numero_registro, codigo_formato, columna_idx, unidad_captura, subcuenta, signo, valor):
    num_registro_str = str(numero_registro).zfill(5)
    tipo_registro = "4"
    codigo_formato_str = str(codigo_formato).zfill(3)
    columna_str = str(columna_idx).zfill(2)
    unidad_captura_str = str(int(float(unidad_captura))).zfill(2)
    subcuenta_str = str(int(float(subcuenta))).zfill(3)
    valor_str = f"{int(valor):020d}"  # Valor sin decimales, 20 caracteres con ceros a la izquierda
    return f"{num_registro_str}{tipo_registro}{codigo_formato_str}{columna_str}{unidad_captura_str}{subcuenta_str}{signo}{valor_str}"

# Obtener la fecha correcta (mes vencido)
fecha_actual = datetime.now()
fecha_mes_vencido = fecha_actual.replace(day=1) - timedelta(days=1)
fecha_str = fecha_mes_vencido.strftime("%d%m%Y")

# Encabezados fijos de la primera y segunda línea
registro_inicial_251 = f"00001114000001{fecha_str}00045SVIDCOLSECS0128"
segunda_linea_251 = "00002" + "2000001100000"

# Inicializar el número de registro y la lista para almacenar las líneas formateadas
numero_registro = 3
txt_datos = []

# Incluir las líneas iniciales del formato 251
txt_datos.append(registro_inicial_251)
txt_datos.append(segunda_linea_251)

# Procesar el formato 251, incluyendo valores en cero
columnas_datos = ['HOMBRES', 'MUJERES', 'HOMBRES_2', 'MUJERES_2', 'TOTAL']
for idx, columna in enumerate(columnas_datos, start=1):
    for _, row in Plantilla_251.iterrows():
        subcuenta = str(row[columna_subcuenta]).strip()
        unidad_captura = str(row[columna_unidad_captura]).strip()
        valor = row[columna]

        # Saltar si el valor es NaN
        if pd.isna(valor):
            continue

        # Determinar el signo del valor
        try:
            valor_float = float(valor)
            signo = '+' if valor_float >= 0 else '-'
            valor_abs = abs(valor_float)

            # Formatear la línea específica para el Formato 251
            linea = formatear_linea(numero_registro, 251, idx, unidad_captura, subcuenta, signo, valor_abs)
            txt_datos.append(linea)
            numero_registro += 1

        except ValueError:
            pass

# Procesar el formato 253, incluyendo valores en cero
for _, row in Plantilla_253.iterrows():
    subcuenta = str(row['SUBCUENTA']).strip()
    unidad_captura = str(row['UNIDAD DE CAPTURA']).strip()
    valor = row['VALOR']

    # Saltar si el valor es NaN
    if pd.isna(valor):
        continue

    # Determinar el signo del valor
    try:
        valor_float = float(valor)
        signo = '+' if valor_float >= 0 else '-'
        valor_abs = abs(valor_float)

        # Formatear la línea específica para el Formato 253
        linea = formatear_linea(numero_registro, 253, 1, unidad_captura, subcuenta, signo, valor_abs)
        txt_datos.append(linea)
        numero_registro += 1

    except ValueError:
        pass

# Agregar la última línea con el conteo de registros y un "5" al final
total_registros = len(txt_datos) + 1
linea_final = f"{str(total_registros).zfill(5)}5"
txt_datos.append(linea_final)

# Guardar el archivo TXT final unificado con los formatos 251 y 253
output_path = "4CUER.txt"
with open(output_path, "w") as f:
    for linea in txt_datos:
        f.write(f"{linea}\n")
